In [39]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as io
import pandas as pd
from pandas import DataFrame as dframe

In [40]:
np.seterr(divide='ignore', invalid='ignore')
#浮動小数点の計算での決め事
#ignoreにすることでエラーを無視する
#divideは0で除算した時どうするか、invalidは浮動小数点演算が無効な場合の処理
data = np.load('A01T.npz')
#numpy独自のバイナリファイルで保存する
data.files
prop = np.c_[data['etyp'], data['epos'], data['edur']]
pd.options.display.max_rows = None
pd.set_option('display.max_rows', None)
#上のコードとしていることはほどんど同じ
dframe(prop, columns=['etype', 'epos','edur']);
#DataFrameに結合配列を格納
#propというデータの3つの列を結合させてDataFrameに格
sample_data = dframe(data['s'])
sample_data = sample_data.iloc[:, 0:22]
#行すべてと22列を切りだしている
#EOGのデータを抜いて、脳波だけを取り出している
sample_data.head();
idx_left = np.argwhere(prop[:, 0]==769)
idx_right = np.argwhere(prop[:, 0]==770)
#イベントタイプが左、右のデータを取得する
#argwhereは配列から条件にあうデータを取得する
idx_left = idx_left.flatten()
idx_right = idx_right.flatten()
#多次元配列を1次元に変換
left_pos = prop[idx_left, 1]
right_pos = prop[idx_right, 1]
#対応するイベントタイプのポジションを取得
left_dur = prop[idx_left, 2]
right_dur = prop[idx_right, 2]
#各イベントの継続時間を取得
dur = 313
E_left = np.zeros([left_pos.shape[0], dur, sample_data.shape[1]])
E_right = np.zeros([right_pos.shape[0], dur, sample_data.shape[1]])
#各イベントのサンプルを取得するための配列形成
for i in range(E_left.shape[0]):
    E_left[i]=sample_data.iloc[left_pos[i]:left_pos[i]+dur, :]
for i in range(E_right.shape[0]):
    E_right[i]=sample_data.iloc[right_pos[i]:right_pos[i]+dur, :]
#対応しているデータを切りだす
#左手や右手の開始時刻から継続時間までのデータを取得する
#1回の実験で22チャンネルのデータが入っている
Xcol = []
for i in range(1, 23):
    if i < 11:
        Xcol.append('EEG0'+str(i))
    else:
        Xcol.append('EEG'+str(i))
#行列の名前を作成
E_left = np.transpose(E_left, (0, 2, 1))
E_right = np.transpose(E_right, (0, 2, 1))
#行と列を変換
ntrain = 60
E_left_tr = E_left[:ntrain, :, :]
E_right_tr = E_right[:ntrain, :, :]
print('Take 54 data of each left and right data as training data\n')
print('left_tr has dimension of: ', E_left_tr.shape)
print('right_tr has dimension of: ', E_right_tr.shape)
#左右各54個ずつのデータを訓練に使用
ntest = 12
E_left_te = E_left[ntrain:ntrain+ntest, :, :]
E_right_te = E_right[ntrain:ntrain+ntest, :, :]
print('Take 18 data of each left and right data as train data\n')
print('left_te has dimension of: ', E_left_te.shape)
print('right_te has dimension of: ', E_right_te.shape)
#左右各18個ずつのデータをテストに使用
nel = 22
#要素数の定義

Take 54 data of each left and right data as training data

left_tr has dimension of:  (60, 22, 313)
right_tr has dimension of:  (60, 22, 313)
Take 18 data of each left and right data as train data

left_te has dimension of:  (12, 22, 313)
right_te has dimension of:  (12, 22, 313)


In [41]:
#各試行のデータをひとまとめに
for i in range(ntrain):
    if i == 0:
        E_left_one = np.squeeze(E_left_tr[i,:,:])
    else:
        E_left_one = np.concatenate([E_left_one,np.squeeze(E_left_tr[i,:,:])],1)
for i in range(ntrain):
    if i == 0:
        E_right_one = np.squeeze(E_right_tr[i,:,:])
    else:
        E_right_one = np.concatenate([E_right_one,np.squeeze(E_right_tr[i,:,:])],1)
print(E_left_one.shape)

(22, 18780)


In [42]:
#変更可能(変更済み)
#各試行の空間共分散を計算
print('Compute covariance of E_left_tr and E_right_tr...\n')
Cov_left_tr = np.zeros((nel, nel))
Cov_right_tr = np.zeros((nel, nel))
Cov_left_sum = np.zeros((nel))
Cov_right_sum = np.zeros((nel))
Cov_left_average = np.zeros((nel))
Cov_right_average = np.zeros((nel))

#その試行での各電極の平均値を求める
for i in range(nel):
    Cov_left_sum[i] = np.sum(E_left_one[i])               
    Cov_left_average[i] = Cov_left_sum[i]/(ntrain*dur)
#各平均値を引く
    E_left_one[i] = E_left_one[i] - Cov_left_average[i]

Cov_left_tr = ((E_left_one@E_left_one.T)/(ntrain*dur))

#その試行での各電極の平均値を求める
for i in range(nel):
    Cov_right_sum[i] = np.sum(E_right_one[i])               
    Cov_right_average[i] = Cov_right_sum[i]/(ntrain*dur)
#各平均値を引く
    E_right_one[i] = E_right_one[i] - Cov_right_average[i]

Cov_right_tr = ((E_right_one@E_right_one.T)/(ntrain*dur))

#.Tは転置を表す、@は行列の積算の演算子、np.traceは対角和

Compute covariance of E_left_tr and E_right_tr...



In [43]:
#空間共分散の合成
print('Calculating composite covariance...\n')
avgCov_c_tr = Cov_left_tr + Cov_right_tr
print('Size of avgCov_c_tr = ', avgCov_c_tr.shape)

Calculating composite covariance...

Size of avgCov_c_tr =  (22, 22)


In [44]:
λ, V = np.linalg.eig(avgCov_c_tr)
#linalg.eigは初めに固有値を含んだ配列を返し、次に固有値ベクトルを返す
#U=Vである
#Vは直行行列でないのでシュミレット直行化法の使用を検討
#もしくは長さ1に規格化
#変更可能
λ_dsc = np.sort(λ)[::-1]
idx_dsc = np.argsort(λ)[::-1]
V_dsc = V[:, idx_dsc]
#[::-1]で逆順になるので降順に並べ替えられる
#argsortは降順になるようなインデントの配列を返す
#λの固有値に対応するように固有値ベクトルVを並び替える

λ_dsc = np.diag(λ_dsc)
#固有値を対角線に配置することで対角化を行った状態にする

#白色化を行う(平均を0、分散を1にする)ことで単位行列を求める
#分散共分散行列に求めた固有値ベクトルと対角化した行列をかける
print('Computing white transformation matrix....\n')
λ_dsc_sqr = np.sqrt(np.linalg.inv(λ_dsc))
#linalg.invは逆行列を返す
P = (λ_dsc_sqr)@(V_dsc.T.T)
#逆行列のルートに固有ベクトルの行列を乗算
print('White transfomartion has dimension of: ', P.shape)

#変更可能
temp = P@avgCov_c_tr@P.T
#対角化した行列が単位行列と等しいか確認
#ちゃんと単位行列になっていない
#正規直交基底になるようにシュミレットを使えば単位行列になるはず

#白色化を行う
# Sl = P Cl P'
S_left = P@Cov_left_tr@P.T
# Sr = P Cr P'
S_right = P@Cov_right_tr@P.T

#白色化を行った行列から左と右の固有値行列を引き出す
λl, Bl = np.linalg.eig(S_left)
λr, Br = np.linalg.eig(S_right)

#昇順にソートを行う
idx_asc = λl.argsort() 

λl_asc = λl[idx_asc]

# 降順にソートを行う
idx_dsc = λr.argsort()[::-1] # Use this index to sort eigenvector largest -> smallest

λr_dsc = λr[idx_dsc]

# 左固有ベクトルを昇順でソート
Bl = Bl[:, idx_asc]
# 右固有ベクトルを降順にソート
Br = Br[:, idx_dsc]

# 固有値ベクトルに固有値行列をかけることで射影行列を取得
W1 = (Br.T@P).T
W2 = (Bl.T@P).T
W = W1

Computing white transformation matrix....

White transfomartion has dimension of:  (22, 22)


In [45]:
#変更可能
# 論文の手順に従い、m = 2に設定
# 2m = 4
# したがって、行列の最初と最後の行の最初のmを取得します
#一方のクラスにおける固有値が大きい初めの𝑚行と他方のクラスにおける固有値が大きい最後の𝑚行を抜き出して新たに行列を生成
#つまり、各クラスの特徴が顕著に出ている最初のｍ行と最後のｍ行を抜き出す
# 残りの行列を削除します
#m=1に変更
m = 2
x, y = W.shape
W_new = np.delete(W, np.s_[m+1:x-m+1], 0)
#np.deleteで行列を削除、np.s[:]で場所を指定、第三要素で軸指定

In [46]:
#ここから先は変更が必要かもしれない
# E を 72 x 22 x 750 の次元に転置
#第2要素が22でなければ変更することはわかるがなぜそれが必要なのかは不明
if E_left_tr.shape[1] != 22 :
    E_left_tr = np.transpose(E_left_tr, (0, 2, 1))
if E_right_tr.shape[1] != 22 :
    E_right_tr= np.transpose(E_right_tr, (0, 2, 1))
    
#新たにZを格納する配列を作成
Z_left_tr = np.zeros((E_left_tr.shape[0], W_new.shape[0], E_left_tr.shape[2]))
Z_right_tr = np.zeros((E_right_tr.shape[0], W_new.shape[0], E_right_tr.shape[2]))

#左のクラス
#射影行列を各試行の脳波に適用する
#得られる行列Zは試行数(今回は54)×2m×試行時間(今回は313)の次元となる
for i in range(Z_left_tr.shape[0]):
    Z_left_tr[i] = W_new@E_left_tr[i]

#右のクラス
for i in range(Z_right_tr.shape[0]):
    Z_right_tr[i] = W_new@E_right_tr[i]
    
#特徴ベクトルを格納するための配列を作成
feat_l_tr =  np.zeros((Z_left_tr.shape[0], Z_left_tr.shape[1], 1))
feat_r_tr =  np.zeros((Z_right_tr.shape[0], Z_right_tr.shape[1], 1))

#式の部分は変更可能
#特徴ベクトルを求める
#左のクラス
for i in range(Z_left_tr.shape[0]):
    var = np.var(Z_left_tr[i], ddof=1, axis=1)[:, np.newaxis]
    varsum = np.sum(var)
    feat_l_tr[i] = np.log(var/varsum);
#np.varで分散値を計算
#axisでどの軸に沿って分散を計算するか求める
#ddofで平均との偏差の2乗の和を(N-ddof)で割り、初期値ではデータ数であるNで割る。ddof=1では不偏分散を求める(つまりN-1)。
#np.newaxisによって新たにサイズが1の次元を追加する。


#右のクラス
for i in range(Z_right_tr.shape[0]):
    var = np.var(Z_right_tr[i], ddof=1, axis=1)[:, np.newaxis]
    varsum = np.sum(var)
    feat_r_tr[i] = np.log(var/varsum);

In [47]:
#Eを20 x 22 x 313に変更
if E_left_te.shape[1] != 22 :
    E_left_te = np.transpose(E_left_te, (0, 2, 1))
if E_right_te.shape[1] != 22 :
    E_right_te = np.transpose(E_right_te, (0, 2, 1))
    
#新たにZを格納する配列を作成
Z_left_te = np.zeros((E_left_te.shape[0], W_new.shape[0], E_left_te.shape[2]))
Z_right_te = np.zeros((E_right_te.shape[0], W_new.shape[0], E_right_te.shape[2]))

#左のクラス
#射影行列を各試行の脳波に適用する
#得られる行列Zは試行数(今回は18)×2m×試行時間(今回は313)の次元となる
for i in range(Z_left_te.shape[0]):
    Z_left_te[i] = W_new@E_left_te[i]

#右のクラス
for i in range(Z_right_te.shape[0]):
    Z_right_te[i] = W_new@E_right_te[i]
    
#特徴ベクトルを格納するための配列を作成
feat_l_te =  np.zeros((Z_left_te.shape[0], Z_left_te.shape[1], 1))
feat_r_te =  np.zeros((Z_right_te.shape[0], Z_right_te.shape[1], 1))


#forのループ部分はおそらく行の4でするべき
#式の部分は変更可能
#特徴ベクトルを求める
#左のクラス
for i in range(Z_left_te.shape[0]):
    var = np.var(Z_left_te[i], ddof=1, axis=1)[:, np.newaxis]
    varsum = np.sum(var)
    feat_l_te[i] = np.log(var/varsum);
#np.varで分散値を計算
#axisでどの軸に沿って分散を計算するか求める
#ddofで平均との偏差の2乗の和を(N-ddof)で割り、初期値ではデータ数であるNで割る。ddof=1では不偏分散を求める(つまりN-1)。
#np.newaxisによって新たにサイズが1の次元を追加する。

#右のクラス
for i in range(Z_right_te.shape[0]):
    var = np.var(Z_right_te[i], ddof=1, axis=1)[:, np.newaxis]
    varsum = np.sum(var)
    feat_r_te[i] = np.log(var/varsum);
    

In [59]:
#まず、特徴ベクトルを2次元に変換
feat_l_tr = np.squeeze(feat_l_tr);
feat_r_tr = np.squeeze(feat_r_tr);
feat_tr = np.concatenate([feat_l_tr, feat_r_tr],0)
feat_tr = feat_tr.T
#まず、特徴ベクトルを2次元に変換
feat_l_te = np.squeeze(feat_l_te);
feat_r_te = np.squeeze(feat_r_te);
feat_l_te = feat_l_te.T
feat_r_te = feat_r_te.T

In [60]:
print(feat_tr.shape,feat_l_te.shape)

(4, 120) (4, 12)


In [63]:
def SRC(feat_l_tr,fea_r_tr,feat_tr,feat_l_te): #rightの時は(feat_r_tr,feat_tr,feat_r_te)
    import cvxpy as cv
    ε=0.3
    SRC_Cla = np.zeros((feat_tr.shape[1], feat_l_te.shape[1]))
    SRC_l_Cla = np.zeros((feat_l_tr.shape[0], feat_l_te.shape[1]))
    SRC_r_Cla = np.zeros((feat_l_tr.shape[0], feat_l_te.shape[1]))  #(feat_r_te.shape[0], feat_r_te.shape[1])でもいい
    answer = np.zeros(feat_l_te.shape[1])
    for i in range(feat_l_te.shape[1]):
        x = cv.Variable((feat_tr.shape[1],1)) #目的関数の次元
        objective = cv.Minimize(cv.norm(x,1))
        constraints = [(cv.norm(feat_l_te[:,i,np.newaxis] - feat_tr@x,2)**2)<=ε]
        prob = cv.Problem(objective, constraints)
        result = prob.solve()
        #classVariableから値を取るためには.valueを付ける
        SRC_Cla[:,i] = np.squeeze(x.value)
        #パラメータxを各クラスのパラメータに分ける
        SRC_l_Cla[:,i] = SRC_Cla[0:feat_l_tr.shape[0],i]
        SRC_r_Cla[:,i] = SRC_Cla[feat_l_tr.shape[0]:SRC_Cla.shape[0],i]
        #分類では
        #配列xの内、右のクラスのみに関する配列を使用して作成したy(右)、左のクラスのみに関する配列を使用して作成したy(左)の
        #残差が小さい方のクラスを割り当てる
        
    #パラメータを使って擬似的に各クラスのテスト信号の特徴ベクトルを作成
    dout_feat_l_te = np.zeros((feat_l_te.shape[0], feat_l_te.shape[1]))
    dout_feat_r_te = np.zeros((feat_l_te.shape[0], feat_l_te.shape[1])) #(feat_r_te.shape[0], feat_r_te.shape[1])でもいい
    for i in range(feat_l_te.shape[1]):
        dout_feat_l_te[:,i] = feat_l_tr.T@SRC_l_Cla[:,i]
    for i in range(feat_r_te.shape[1]):
        dout_feat_r_te[:,i] = feat_r_tr.T@SRC_r_Cla[:,i]
    
    #作成した擬似特徴ベクトルとの残差を求める
    R_l =  np.zeros((feat_l_te.shape[0], feat_l_te.shape[1])) #残差を格納するための配列
    R_r =  np.zeros((feat_l_te.shape[0], feat_l_te.shape[1])) #(feat_r_te.shape[0], feat_r_te.shape[1])でもいい
    for i in range(feat_l_te.shape[1]):
        R_l[:,i] = feat_l_te[:,i]-dout_feat_l_te[:,i]
        R_r[:,i] = feat_l_te[:,i]-dout_feat_r_te[:,i]
        if np.linalg.norm(R_l[:,i], ord=2) < np.linalg.norm(R_r[:,i], ord=2):
            answer[i] = 1
        elif np.linalg.norm(R_l[:,i], ord=2) > np.linalg.norm(R_r[:,i], ord=2):
            answer[i] = 2
    
    return answer

In [65]:
answer = SRC(feat_l_tr,feat_r_tr,feat_tr,feat_r_te)
x = 0
for i in range(answer.shape[0]):
    if answer[i] == 2:
        x = x + 1
print((x/answer.shape[0])*100)

100.0
